In [1]:
import json
import time
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

drop = False

In [2]:
# Import library to read ground truth data
bigdama_path = "C:\\Users\\alexc\\Desktop\\SNET\\Githubs\\OutlierDenStreamBD\\"
sys.path.append(bigdama_path)

from readGroundTruth import groundTruth
truth = groundTruth(bigdama_path + 'GrounTruth\\portflap_first.txt', fileType='csv')
for anomaly in truth.events:
    print(anomaly['startTime'], anomaly['endTime'])

1504286090 1504286390
1504286450 1504286760
1504286868 1504287178
1504287230 1504287600


In [3]:
configuration = json.load(open('C:\\Users\\alexc\\Desktop\\SNET\\Githubs\\OutlierDenStreamBD\\configuration.json'))

In [4]:
def normalize_matrix(df):
    return (df - df.mean())/df.std()

In [5]:
def seriesSynchronizer(df):
    out = pd.DataFrame()
    df = df.iloc[:,1:]
    for c in df.columns:
        out = pd.concat([out.reset_index(drop=True), df[c].dropna().reset_index(drop=True)], axis=1)

    out = out.dropna(how="any",axis=0).reset_index(drop=True)
    return out

In [6]:
def staticThreshholdSymbolizer(df,wsize):
    out = df.copy()
    symbols = pd.DataFrame()
    for c in df.columns:
        varMean = df[c].mean()
        r1 = varMean-2*df[c].std()
        r2 = varMean-df[c].std()
        r3 = varMean+df[c].std()
        r4 = varMean+2*df[c].std()
        
        out[c]="A"
        out.loc[df[c]>r1,c]="B"
        out.loc[df[c]>r2,c]="C"
        out.loc[df[c]>varMean,c]="D"
        out.loc[df[c]>r3,c]="E"
        out.loc[df[c]>r4,c]="F"
    
        out2 = []
        for r in range(wsize-1,len(df.index)-wsize):
            temp = ""
            for i in range(wsize):
                temp=temp+out.loc[r+i,c]
            out2 = out2+[temp]
        symbols[c]=out2
    return symbols
        

In [7]:
def getData():
    features = 'CP'

    for dataset in configuration['dataset']['list']:
        telemetry_allLeafs = pd.DataFrame()
        truth = groundTruth(bigdama_path + 'GrounTruth\\'+dataset+'.txt', fileType='csv')

        # Read node dataset
        dff = pd.read_csv(bigdama_path+configuration['dataset']['path']+"leaf1"+dataset+'.csv', low_memory = False)\
                        .dropna()\
                        .drop('Unnamed: 0', axis=1)

        telemetry_allLeafs = dff[[dff.columns[0],dff.columns[1]]].astype('int').copy()
        telemetry_allLeafs2 = telemetry_allLeafs.copy()

        # Iterate on all the nodes chosen in the configuration file
        for node in configuration['nodes']:

            # Read node dataset
            df = pd.read_csv(bigdama_path+configuration['dataset']['path']+node+dataset+'.csv', low_memory = False)\
                            .dropna()\
                            .drop('Unnamed: 0', axis=1)

            #comentar sig 3 lineas para usar todas
            features = configuration['featureList']
            features = pd.concat([pd.Series(['time']), pd.Series(features)])
            df = df[features]
            ####

            df_short = df.loc[:,df.std()!=0]

            dfNormalized = df_short.dropna(axis=1)
            names = []
            for i in df_short.columns:
                names = names+[i+"_"+node]
            names[0]='time'

            dfNormalized2 = dfNormalized.copy()

            dfNormalized.columns = df_short.columns
            dfNormalized2.columns = names

            # Add leaf to allLeafs dataset
            telemetry_allLeafs = pd.merge(telemetry_allLeafs, dfNormalized, on='time', how='outer')
            telemetry_allLeafs2 = pd.merge(telemetry_allLeafs2, dfNormalized2, on='time', how='outer')

    telemetry_allLeafs = telemetry_allLeafs.drop(telemetry_allLeafs.columns[1],axis=1).sort_values(by="time",axis=0)
    telemetry_allLeafs2 = telemetry_allLeafs2.drop(telemetry_allLeafs2.columns[1],axis=1).sort_values(by="time",axis=0)

    telemetry_allLeafs2 = telemetry_allLeafs2.reset_index().drop(['index'],axis=1)
    telemetry_allLeafs2 = seriesSynchronizer(telemetry_allLeafs2)
    
    return telemetry_allLeafs2

In [8]:
def staticThreshholdSymbolizer2(df,divs,wsize):
    out = df.copy()
    symbols = pd.DataFrame()
    for c in df.columns:
        varMean = df[c].mean()
        dev = df[c].std()
        out[c]="A"
        if divs==2: 
            out.loc[df[c]>varMean,c]="B"
        elif divs==3:
            out.loc[df[c]>varMean-dev,c]="B"
            out.loc[df[c]>varMean+dev,c]="C"
        elif divs==4:
            out.loc[df[c]>varMean-dev,c]="B"
            out.loc[df[c]>varMean,c]="C"
            out.loc[df[c]>varMean+dev,c]="D"
        elif divs==5:
            out.loc[df[c]>varMean-2*dev,c]="B"
            out.loc[df[c]>varMean-dev,c]="C"
            out.loc[df[c]>varMean+dev,c]="D"
            out.loc[df[c]>varMean+2*dev,c]="E"
        elif divs==6:
            out.loc[df[c]>varMean-2*dev,c]="B"
            out.loc[df[c]>varMean-dev,c]="C"
            out.loc[df[c]>varMean,c]="D"
            out.loc[df[c]>varMean+dev,c]="E"
            out.loc[df[c]>varMean+2*dev,c]="F"
    
        out2 = []
        temp = "AAAAAA"
        temp = temp[:wsize]
        for r in range(len(df.index)):
            temp=temp[1:]+out.loc[r,c]
            out2 = out2+[temp]
        symbols[c]=out2
    return symbols
        

In [9]:
def dynamicThreshholdSymbolizer(df,wsize,window):
    out = df.copy()
    symbols = pd.DataFrame()
    for c in df.columns:
        out2 = []
        for r in range(wsize-1,len(df.index)-wsize):
            lim1 = max(0,r-window)
            lim2 = r
            varMean = df.loc[lim1:lim2,c].mean()
            r1 = varMean-2*df.loc[lim1:lim2,c].std()
            r2 = varMean-df.loc[lim1:lim2,c].std()
            r3 = varMean+df.loc[lim1:lim2,c].std()
            r4 = varMean+2*df.loc[lim1:lim2,c].std()

            out[c]="A"
            out.loc[df[c]>r1,c]="B"
            out.loc[df[c]>r2,c]="C"
            out.loc[df[c]>varMean,c]="D"
            out.loc[df[c]>r3,c]="E"
            out.loc[df[c]>r4,c]="F"

            temp = ""
            for i in range(wsize):
                temp=temp+out.loc[r+i,c]
            out2 = out2+[temp]
        symbols[c]=out2
    return symbols
        

In [10]:
def dynamicThreshholdSymbolizer2(df,divs,wsize,window):
    out = df.copy()
    symbols = pd.DataFrame()
    for c in df.columns:
        out2 = []
        temp = "AAAAAA"
        temp = temp[:wsize]
        for r in range(len(df.index)):
            lim1 = max(0,r-window)
            lim2 = r
            varMean = df.loc[lim1:lim2,c].mean()
            dev = df.loc[lim1:lim2,c].std()
            
            out[c]="A"
            if divs==2: 
                out.loc[df[c]>varMean,c]="B"
            elif divs==3:
                out.loc[df[c]>varMean-dev,c]="B"
                out.loc[df[c]>varMean+dev,c]="C"
            elif divs==4:
                out.loc[df[c]>varMean-dev,c]="B"
                out.loc[df[c]>varMean,c]="C"
                out.loc[df[c]>varMean+dev,c]="D"
            elif divs==5:
                out.loc[df[c]>varMean-2*dev,c]="B"
                out.loc[df[c]>varMean-dev,c]="C"
                out.loc[df[c]>varMean+dev,c]="D"
                out.loc[df[c]>varMean+2*dev,c]="E"
            elif divs==6:
                out.loc[df[c]>varMean-2*dev,c]="B"
                out.loc[df[c]>varMean-dev,c]="C"
                out.loc[df[c]>varMean,c]="D"
                out.loc[df[c]>varMean+dev,c]="E"
                out.loc[df[c]>varMean+2*dev,c]="F"
    
            temp = temp[1:]+out.loc[r,c]
            out2 = out2+[temp]
        symbols[c]=out2
    return symbols
        

In [11]:
telemetry_allLeafs2 = getData()
#print(telemetry_allLeafs2.count())

In [12]:
if drop:
    samplings = [0]
    for c in telemetry_allLeafs2.columns[1:]:
        samplings = samplings + [telemetry_allLeafs2[c].first_valid_index()]

    telemetry_allLeafs2.loc[-1] = samplings # adding a row
    telemetry_allLeafs2.index = telemetry_allLeafs2.index + 1  # shifting index
    telemetry_allLeafs2 = telemetry_allLeafs2.sort_index()  # sorting by index

    #display(telemetry_allLeafs2.loc[0])

    grupo0 = telemetry_allLeafs2.loc[1:,telemetry_allLeafs2.loc[0,:]==0].drop('time',axis=1).dropna(how="all").reset_index(drop=True)
    grupo1 = telemetry_allLeafs2.loc[1:,telemetry_allLeafs2.loc[0,:]==1].dropna(how="all").reset_index(drop=True)
    grupo2 = telemetry_allLeafs2.loc[1:,telemetry_allLeafs2.loc[0,:]==2].dropna(how="all").reset_index(drop=True)
    grupo3 = telemetry_allLeafs2.loc[1:,telemetry_allLeafs2.loc[0,:]==3].dropna(how="all").reset_index(drop=True)
    grupo4 = telemetry_allLeafs2.loc[1:,telemetry_allLeafs2.loc[0,:]==4].dropna(how="all").reset_index(drop=True)

    print(grupo0.shape)
    print(grupo1.shape)
    print(grupo2.shape)
    print(grupo3.shape)
    print(grupo4.shape)

    telemetry_allLeafs2 = telemetry_allLeafs[1:]
    #display(telemetry_allLeafs2.loc[0])

    #display(grupo0)
    
else:
    #telemetry_allLeafs2 = seriesSynchronizer(telemetry_allLeafs2)
    display(telemetry_allLeafs2)

,active-routes-count_leaf1,deleted-routes-count_leaf1,paths-count_leaf1,protocol-route-memory_leaf1,routes-counts_leaf1,performance-statistics__vrf__inbound-update-messages_leaf1,vrf__network-count_leaf1,vrf__path-count_leaf1,vrf__update-messages-received_leaf1,active-routes-count_leaf2,...,vrf__update-messages-received_spine3,active-routes-count_spine4,deleted-routes-count_spine4,paths-count_spine4,protocol-route-memory_spine4,routes-counts_spine4,performance-statistics__vrf__inbound-update-messages_spine4,vrf__network-count_spine4,vrf__path-count_spine4,vrf__update-messages-received_spine4
0,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
1,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
2,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
3,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
4,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
5,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
6,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
7,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
8,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0
9,183.0,0.0,1194.0,157280.0,184.0,27026.0,389.0,5788.0,27026.0,180.0,...,83573.0,172.0,0.0,1057.0,140400.0,172.0,81185.0,389.0,10442.0,81185.0


In [13]:
if drop:
    #display(staticThreshholdSymbolizer(grupo0.iloc[:,:],3))
    #display(staticThreshholdSymbolizer2(grupo0.iloc[:,:],3,3))
    statsym = staticThreshholdSymbolizer2(grupo0.iloc[:,:],3,3)
else:
    #display(staticThreshholdSymbolizer(telemetry_allLeafs2.iloc[:,:],3))
    #display(staticThreshholdSymbolizer2(telemetry_allLeafs2.iloc[:,:],3,3))
    statsym = staticThreshholdSymbolizer2(telemetry_allLeafs2.iloc[:,5:15],3,3)

In [14]:
if drop:
    #display(dynamicThreshholdSymbolizer(grupo0.iloc[:,:],3,10))
    #display(dynamicThreshholdSymbolizer2(grupo0.iloc[:,:],3,3,10))
    dynsym = dynamicThreshholdSymbolizer2(grupo0.iloc[:,:],3,3,10)
else:
    #display(dynamicThreshholdSymbolizer(telemetry_allLeafs2.iloc[:,:],3,10))
    #display(dynamicThreshholdSymbolizer2(telemetry_allLeafs2.iloc[:,:],3,3,10))
    dynsym = dynamicThreshholdSymbolizer2(telemetry_allLeafs2.iloc[:,5:15],3,3,10)

In [15]:
display(statsym)

,performance-statistics__vrf__inbound-update-messages_leaf1,vrf__network-count_leaf1,vrf__path-count_leaf1,vrf__update-messages-received_leaf1,active-routes-count_leaf2,deleted-routes-count_leaf2,paths-count_leaf2,protocol-route-memory_leaf2,performance-statistics__vrf__inbound-update-messages_leaf2,vrf__path-count_leaf2
0,AAA,AAB,AAB,AAA,AAB,AAB,AAB,AAB,AAA,AAB
1,AAA,ABB,ABB,AAA,ABB,ABB,ABB,ABB,AAA,ABB
2,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
3,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
4,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
5,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
6,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
7,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
8,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB
9,AAA,BBB,BBB,AAA,BBB,BBB,BBB,BBB,AAA,BBB


In [16]:
display(dynsym)

,performance-statistics__vrf__inbound-update-messages_leaf1,vrf__network-count_leaf1,vrf__path-count_leaf1,vrf__update-messages-received_leaf1,active-routes-count_leaf2,deleted-routes-count_leaf2,paths-count_leaf2,protocol-route-memory_leaf2,performance-statistics__vrf__inbound-update-messages_leaf2,vrf__path-count_leaf2
0,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
1,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
2,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
3,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
4,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
5,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
6,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
7,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
8,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA
9,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA


In [17]:
x = dynsym.groupby(by='paths-count_leaf2').count().iloc[:,0]
x.columns="counts"
x

paths-count_leaf2
AAA    223
AAB      8
AAC      4
ABB      9
ACC      4
BAA      9
BAB      1
BBA     10
BBB     50
BBC      3
BCA      2
BCB      1
CAA      3
CBB      4
CCA      1
CCB      3
CCC     12
Name: performance-statistics__vrf__inbound-update-messages_leaf1, dtype: int64

In [18]:
testVar = dynsym['paths-count_leaf2']
testVarNum = testVar.copy()
testVarNum[testVar=='AAA']=1
testVarNum[testVar=='AAB']=2
testVarNum[testVar=='AAC']=3
testVarNum[testVar=='ABA']=4
testVarNum[testVar=='ABB']=5
testVarNum[testVar=='ABC']=6
testVarNum[testVar=='ACA']=7
testVarNum[testVar=='ACB']=8
testVarNum[testVar=='ACC']=9

testVarNum[testVar=='BAA']=10
testVarNum[testVar=='BAB']=11
testVarNum[testVar=='BAC']=12
testVarNum[testVar=='BBA']=13
testVarNum[testVar=='BBB']=14
testVarNum[testVar=='BBC']=15
testVarNum[testVar=='BCA']=16
testVarNum[testVar=='BCB']=17
testVarNum[testVar=='BCC']=18

testVarNum[testVar=='CAA']=19
testVarNum[testVar=='CAB']=20
testVarNum[testVar=='CAC']=21
testVarNum[testVar=='CBA']=22
testVarNum[testVar=='CBB']=23
testVarNum[testVar=='CBC']=24
testVarNum[testVar=='CCA']=25
testVarNum[testVar=='CCB']=26
testVarNum[testVar=='CCC']=27
